# Topic Modeling
#### RIHAD VARIAWA
#### 18-03-2019

The dataset used for this analysis can be found using the following [link](https://www.kaggle.com/therohk/million-headlines)

## About the Dataset:
This includes the entire corpus of articles published over a period of 15 years by the ABC website in the given time range. With a volume of 200 articles per day and a good focus on international news, we can be fairly certain that every event of significance has been captured here.

Digging into the keywords, one can see all the important episodes shaping the last decade and how they evolved over time. Ex: financial crisis, iraq war, multiple US elections, ecological disasters, terrorism, famous people, Australian crimes etc.

## Purpose of this analysis
We will build a statistical model for discovering the abstract “topics” that occur in a collection of documents, using Latent Dirichlet Allocation (LDA) as an example of topic modeling to classify text in a document by a particular topic. It builds a topic per document model and words per topic model, modeled as Dirichlet distributions.

### Import the Data

In [0]:
from google.colab import files
uploaded = files.upload()

Saving abcnews-date-text.csv.zip to abcnews-date-text.csv (1).zip


In [0]:
!unzip abcnews-date-text.csv.zip

Archive:  abcnews-date-text.csv.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of abcnews-date-text.csv.zip or
        abcnews-date-text.csv.zip.zip, and cannot find abcnews-date-text.csv.zip.ZIP, period.


### Loading the library and dataset 

In [0]:
import pandas as pd
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False);
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

In [0]:
# peak at the data
print(len(documents))
print(documents[:5])

### Data Preprocessing
We will perform the following steps:

* Tokenization: Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation.
* Words that have fewer than 3 characters are removed.
* All stopwords are removed.
* Words are lemmatized :  words in third person are changed to first person and verbs in past and future tenses are changed into present.
* Words are stemmed : words are reduced to their root form.

In [0]:
# load gensim and nltk libraries
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

In [0]:
# write a function to perform lemmatize and stem preprocessing steps on dataset
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [0]:
# select a document to preview after preprocessing
doc_sample = documents[documents['index'] == 4310].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

In [0]:
# preprocess the headline text, saving the results as ‘processed_docs’
processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:10]

In [0]:
# bag of words
# create a dictionary from ‘processed_docs’ containing the number of times a word appears in the training set
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

In [0]:
# filter out tokens that appear in
# less than 15 documents (absolute number) or
# more than 0.5 documents (fraction of total corpus size, not absolute number)
# after the above two steps, keep only the first 100000 most frequent tokens
dictionary.filter_extremes(no_below = 15, no_above = 0.5, keep_n = 100000)

In [0]:
# Gensim doc2bow
# for each document we create a dictionary reporting how many
# words and how many times those words appear
# save this to ‘bow_corpus’, then check our selected document earlier
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

In [0]:
# preview Bag Of Words for our sample preprocessed document
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

In [0]:
# TF-IDF
# create tf-idf model object using models.TfidfModel on ‘bow_corpus’ and save it to ‘tfidf’, 
# then apply transformation to the entire corpus and call it ‘corpus_tfidf’
# lastly we preview TF-IDF scores for our first document
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

In [0]:
# running LDA using Bag of Words
Train our lda model using gensim.models.LdaMulticore and save it to ‘lda_model’
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

For each topic, we will explore the words occuring in that topic and its relative weight.

In [0]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Can you distinguish different topics using the words in each topic and their corresponding weights?

In [0]:
# running LDA using TF-IDF
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Again, can you distinguish different topics using the words in each topic and their corresponding weights?

In [0]:
# performance evaluation by classifying sample document using LDA Bag of Words model
# we will check where our test document would be classified
processed_docs[4310]

In [0]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))

Our test document has the highest probability to be part of the topic that our model assigned, which is the accurate classification.

In [0]:
# performance evaluation by classifying sample document using LDA TF-IDF model
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))

Our test document has the highest probability to be part of the topic that our model assigned, which is the accurate classification.

In [0]:
# testing model on unseen document
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))